<a href="https://colab.research.google.com/github/emersonrafaels/base360_restricoes/blob/main/Codigo_Restri%C3%A7%C3%B5es.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pydantic

In [2]:
import pandas as pd
from pydantic import BaseModel, ValidationError, conlist

In [3]:
from pydantic import BaseModel, ValidationError, model_validator
from typing import Optional, Callable, List, Tuple
import pandas as pd

class DataFrameConditionModel(BaseModel):
    """
    Modelo de validação de dados para condições aplicadas em um DataFrame.

    Este modelo valida que o DataFrame e as condições fornecidas são válidos
    antes de aplicar as condições ao DataFrame.

    Atributos:
        df (pd.DataFrame): O DataFrame a ser validado.
        conditions (List[Tuple[Callable[[pd.Series], bool], str]]):
            Lista de condições onde cada condição é uma função (lambda) e um nome de coluna.
    """
    # O df DEVE SER UM DATAFRAME
    df: pd.DataFrame

    """
      A CONDIÇÃO DEVE SER UMA LISTA DE TUPLAS, SENDO QUE CADA ELEMENTO
      DA TUPLA DEVE SER UM CALLABLE E O NOME DA COLUNA EM STRING

      Callable[[pd.Series], bool]:

      Callable: Refere-se a qualquer objeto que pode ser chamado como uma função.
                Em outras palavras, ele aceita funções ou qualquer objeto que implemente o método __call__.

                [pd.Series]: Define que a função aceita um argumento do tipo pd.Series.
                pd.Series é um objeto do pandas que representa uma coluna de dados, ou seja, uma série unidimensional de dados indexados.

                bool: Indica que a função (ou Callable) deve retornar um valor booleano, ou seja, True ou False.

                Em resumo, Callable[[pd.Series], bool] significa que a função espera uma pd.Series como entrada e retorna um valor booleano.

    """
    conditions: List[Tuple[Callable[[pd.Series], bool], str]]

    class Config:
        arbitrary_types_allowed = True

    @model_validator(mode='before')
    def check_conditions(cls, values):
        """
        Valida as condições fornecidas.

        Garante que a lista de condições seja válida e que cada condição seja
        uma função e o nome da coluna seja uma string.

        Parâmetros:
            values (dict): O dicionário de valores a serem validados.

        Retorna:
            dict: Os valores validados.

        Levanta:
            ValueError: Se as condições não forem válidas.
        """
        conditions = values.get("conditions")
        if not conditions or not isinstance(conditions, list):
            raise ValueError("A lista de condições deve ser fornecida e deve ser uma lista válida.")
        for condition, column_name in conditions:
            if not callable(condition):
                raise ValueError("Cada condição deve ser uma função (callable).")
            if not isinstance(column_name, str):
                raise ValueError("O nome da coluna deve ser uma string.")
        return values

class DataFrameManipulatorBase:
    """
    Classe base para manipulação de DataFrames.

    Fornece métodos para adicionar colunas ao DataFrame com base em condições.

    Atributos:
        df (pd.DataFrame): O DataFrame a ser manipulado.
    """

    def __init__(self, df):
        """
        Inicializa a classe base com o DataFrame.

        Parâmetros:
            df (pd.DataFrame): O DataFrame onde as condições serão aplicadas.
        """
        self.df = df

    def add_condition_column(self, condition: Callable[[pd.Series], bool], column_name: str):
        """
        Adiciona uma nova coluna ao DataFrame com base em uma condição.

        Parâmetros:
            condition (Callable[[pd.Series], bool]):
                Função lambda que retorna True ou False, indicando se a condição é atendida.
            column_name (str): O nome da coluna onde o resultado será salvo.
        """
        self.df[column_name] = self.df.apply(lambda row: 'Sim' if condition(row) else 'Não', axis=1)

    def get_dataframe(self):
        """
        Retorna o DataFrame atualizado com as colunas adicionadas.

        Retorna:
            pd.DataFrame: O DataFrame atualizado.
        """
        return self.df

class DataFrameConditioner(DataFrameManipulatorBase):
    """
    Classe para aplicar condições ao DataFrame com validação de dados.

    Esta classe valida os dados de entrada usando Pydantic e aplica as condições ao DataFrame,
    adicionando colunas com os resultados.

    Atributos:
        conditions (List[Tuple[Callable[[pd.Series], bool], str]]):
            Lista de condições validadas onde cada condição é uma função (lambda) e um nome de coluna.
    """

    def __init__(self, df, conditions):
        """
        Inicializa a classe com validação dos dados e aplica as condições ao DataFrame.

        Parâmetros:
            df (pd.DataFrame): O DataFrame onde as condições serão aplicadas.
            conditions (List[Tuple[Callable[[pd.Series], bool], str]]):
                Lista de condições onde cada condição é uma função (lambda) e um nome de coluna.
        """

        # VALIDANDO OS DADOS
        model = DataFrameConditionModel(df=df, conditions=conditions)

        # INICIALIZANDO A CLASSE MANIPULATORBASE
        super().__init__(model.df)

        # SALVANDO AS CONDIÇÕES
        self.conditions = model.conditions

    def apply_conditions(self):
        """
        Aplica todas as condições validadas ao DataFrame.

        Adiciona novas colunas ao DataFrame com os resultados de cada condição.
        """

        # PARA CADA CONDIÇÃO
        for condition, column_name in self.conditions:

          # EXECUTANDO A CLASSE RESPONSÁVEL POR MANIPULAR A BASE
          self.add_condition_column(condition, column_name)


In [4]:
# Exemplo de uso da classe derivada
df = pd.DataFrame({
    'Nome': ['Ana', 'Bruno', 'Carlos', 'Daniela', 'Eduardo'],
    'Idade': [25, 32, 40, 29, 35],
    'Cidade': ['São Paulo', 'Rio de Janeiro', 'Belo Horizonte', 'Curitiba', 'Porto Alegre'],
    'Salário': [3000, 4000, 2500, 5000, 4500]
})

In [5]:
conditions = [
    (lambda row: row['Idade'] > 30, 'Idade_Maior_30'),
    (lambda row: row['Salário'] > 4000, 'Salario_Maior_4000'),
    (lambda row: row['Cidade'] == 'São Paulo', 'Cidade_SP'),
]

In [6]:
# Instancia a classe e aplica as condições
try:
    conditioner = DataFrameConditioner(df, conditions)
    conditioner.apply_conditions()
    df_result = conditioner.get_dataframe()
except ValidationError as e:
    print(e.json())

In [7]:
df_result

,Nome,Idade,Cidade,Salário,Idade_Maior_30,Salario_Maior_4000,Cidade_SP
0,Ana,25,São Paulo,3000,Não,Não,Sim
1,Bruno,32,Rio de Janeiro,4000,Sim,Não,Não
2,Carlos,40,Belo Horizonte,2500,Sim,Não,Não
3,Daniela,29,Curitiba,5000,Não,Sim,Não
4,Eduardo,35,Porto Alegre,4500,Sim,Sim,Não
